# Plateforme Agnostique de Traitement et d'Analyse des Textes
### Carnet d'expérimentation
---

## Sujet : Analyse SSO


---

# Initialisations
---

In [1]:
_rs = 42

In [2]:
cd ../..

/Volumes/Geek/Work/Patat


In [4]:
import ast
import importlib
import numpy as np
import pandas as pd
import torch
import seaborn as sns
from tqdm.notebook import tqdm,trange
tqdm.pandas()

# Préparation Données

---

## Chargement

In [60]:
cd Work/Patat

/Volumes/Geek/Work/Patat


In [5]:
import patat.util.file

filename = 'data/prod/230517-OIDS-Label.pickle'

df_label = patat.util.file.pickle_load(filename)

In [27]:
df_corpus = pd.read_csv('data/prod/230510-OIDS-Corpus.csv')

In [26]:
df_corpus_new = pd.read_pickle('data/tmp/230526-Corpus.pickle')

In [13]:
df_label.value_counts('site')

site
www.francesoir.fr                    169
www.francetvinfo.fr                   91
www.breizh-info.com                   66
www.wikistrike.com                    62
lezarceleurs.blogspot.com             58
lesmoutonsrebelles.com                47
lemediaen442.fr                       32
www.profession-gendarme.com           28
lesdeqodeurs.fr                       28
fr.sott.net                           26
www.dreuz.info                        25
www.lelibrepenseur.org                23
www.polemia.com                       19
reseauinternational.net               17
actu.fr                               17
www.mondialisation.ca                 16
www.nouvelordremondial.cc             14
lesakerfrancophone.fr                 13
www.lesalonbeige.fr                   13
www.voltairenet.org                   12
lesobservateurs.ch                     9
www.anguillesousroche.com              9
lecourrier-du-soir.com                 9
www.cnews.fr                           9
www.preuves

In [14]:
df_sso = pd.DataFrame(df_label.value_counts('site'))

In [31]:
df_corpus = pd.concat([df_corpus,df_corpus_new])

In [32]:
df_corpus

,Unnamed: 0,url,site,title,author,date,tags,paragraphs,date_iso,abstract,c_count,p_count,url_h,text,predict_parags,predict_infox,predict_parags_json,article,parse_method
0,0.0,https://www.alnas.fr/actualite/en-vrac/les-emi...,www.alnas.fr,Les Émirats arabes unis et l’Arabie saoudite f...,Par Sarah -,03/01/2023,['ActualitésEn vrac'],['Plus des trois quarts des importations japon...,2023-01-03T00:00:00,NaN,1738.0,6.0,d9726057bcbe0182c5bec27f5cc2e1c73594a361,Les Émirats arabes unis et l’Arabie saoudite f...,[0.00455141 0.00462569 0.0124964 0.02172739 0...,0.166,"[0.004551408922451928, 0.004625692063235327, 0...",NaN,NaN
1,1.0,https://www.anguillesousroche.com/changement-c...,www.anguillesousroche.com,Article d’opinion du NY Times : « S’accoupler ...,NaN,4 janvier 2023,['Changement climatique'],['Un article d’opinion du New York Times suggè...,2023-01-04T00:00:00,« Nous serons plus nombreux à pouvoir monter d...,3420.0,23.0,7dce1f2479231f6e9606c69da67444b634c2d32d,Article d’opinion du NY Times : « S’accoupler ...,[0.04293733 0.01524119 0.01100111 0.00320901 0...,0.249,"[0.04293733084455618, 0.015241186019098697, 0....",NaN,NaN
2,2.0,https://www.bvoltaire.fr/prediction-pour-2023-...,www.bvoltaire.fr,Prédiction pour 2023 : ça va bien se passer… j...,Frédéric Lassez,3 janvier 2023,['Articles\nPolitique\nVoeux Macron'],"['Le 31 décembre dernier, Emmanuel Macron étai...",2023-01-03T00:00:00,NaN,4521.0,10.0,b11055abd5b8f6c385a79096ae29d90f8a0348ff,Prédiction pour 2023 : ça va bien se passer… j...,[0.09663327 0.01465098 0.0102211 0.03372282 0...,0.137,"[0.09663327321162296, 0.014650982206132444, 0....",NaN,NaN
3,3.0,https://lesdeqodeurs.fr/le-premier-ministre-or...,lesdeqodeurs.fr,Le Premier ministre Orbán affirme que le scand...,NaN,19/12/2022,"['Andrea Cozzolino', 'Bruxelles', 'corruption'...",['Il est temps d’assécher le marécage de Bruxe...,2022-12-19T00:00:00,NaN,3668.0,10.0,790fa6da6e1d9151e0ad6ce08627f738ad1e864c,Le Premier ministre Orbán affirme que le scand...,[0.02769065 0.01691544 0.06271069 0.01470729 0...,0.171,"[0.027690645170104517, 0.01691544245781248, 0....",NaN,NaN
4,4.0,https://lesobservateurs.ch/2022/12/18/herault-...,lesobservateurs.ch,Hérault : 300 musulmans armés de kalachnikovs ...,Francis Gruzelle,Article publié le 18.12.2022,[],"['« La horde sauvage » nouvelle version, sans ...",2022-12-21T00:35:39+01:00,NaN,6828.0,13.0,2b314df4e0710050ae0441d6423177362127c784,Hérault : 300 musulmans armés de kalachnikovs ...,[0.6599003 0.77176296 0.60872019 0.19192202 0...,0.736,"[0.6599003003401536, 0.7717629633228862, 0.608...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,NaN,https://www.alnas.fr/actualite/politique/onu-u...,www.alnas.fr,ONU : Un tiers de la population arabe vit dans...,Par Maryam -,02/01/2023,"[Accueil, Actualités, Politique]",[Un tiers de la population arabe vit sous le s...,2023-01-02T00:00:00,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Un tiers de la population arabe vit sous le se...,parse
277,NaN,https://www.alnas.fr/actualite/sport-actualite...,www.alnas.fr,Coupe du Monde : Les joueurs marocains se pros...,Par Oussama -,27/11/2022,"[Accueil, Actualités, Sport]",[Le Maroc a battu la Belgique 2-0 lors du matc...,2022-11-27T00:00:00,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Le Maroc a battu la Belgique 2-0 lors du match...,parse
278,NaN,https://www.alnas.fr/actualite/buzz/trop-de-no...,www.alnas.fr,« Trop de noirs et de musulmans » : le PSG ouv...,Par Youcef -,12/04/2023,"[Accueil, Actualités, Buzz]",[Quelques heures après les révélations de Juli...,2023-04-12T00:00:00,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quelques heures après les révélations de Julie...,parse
279,NaN,https://www.alnas.fr/actualite/sport-actualite...,www.alnas.fr,Coupe du Monde 2030 : Le roi Mohamed VI annonc...,Par Ayyoub -,16/03/2023,"[Accueil, Actualités, Sport]","[L’idée était dans l’air depuis quelque temps,...",2023-03-16T00:00:00,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"L’idée était dans l’air depuis quelque temps, ...",parse


In [17]:
df_sso=df_sso.rename(columns={'count':'recueil'})

In [36]:
df_corpus_sites = pd.DataFrame(df_corpus.value_counts('site')).rename(columns={'count':'corpus'})

In [38]:
df_sites = pd.concat([df_sso,df_corpus_sites],axis=1).fillna(0).sort_values('recueil',ascending=False)

In [41]:
df_sites = df_sites.astype(int)

In [42]:
import requests

In [50]:
def crawlable(site):
    url = f'https://{site}/'
    print(f'Testing {site}')
    r = requests.get(url)
    return r.status_code

In [51]:
df_sites['http_code']=df_sites.index.map(crawlable)

Testing www.francesoir.fr
Testing www.francetvinfo.fr
Testing www.breizh-info.com
Testing www.wikistrike.com
Testing lezarceleurs.blogspot.com
Testing lesmoutonsrebelles.com
Testing lemediaen442.fr
Testing www.profession-gendarme.com
Testing lesdeqodeurs.fr
Testing fr.sott.net
Testing www.dreuz.info
Testing www.lelibrepenseur.org
Testing www.polemia.com
Testing actu.fr
Testing reseauinternational.net
Testing www.mondialisation.ca
Testing www.nouvelordremondial.cc
Testing lesakerfrancophone.fr
Testing www.lesalonbeige.fr
Testing www.voltairenet.org
Testing lesobservateurs.ch
Testing www.anguillesousroche.com
Testing lecourrier-du-soir.com
Testing www.cnews.fr
Testing infodujour.fr
Testing www.preuvesduparanormal.fr
Testing www.les-crises.fr
Testing fr.novopress.info
Testing www.alnas.fr
Testing www.medias-presse.info
Testing www.bvoltaire.fr
Testing ripostelaique.com
Testing lalettrepatriote.com
Testing theconversation.com
Testing lumieresurgaia.com
Testing www.la-petite-souris-normande

In [53]:
df_sites['crawlable']=df_sites['http_code'].apply(lambda status: status==200)

In [55]:
df_sites['url']=df_sites.index.map(lambda site: f'https://{site}/')

In [61]:
from patat.imp.parser_params import url_masks

In [64]:
page_masks = []
for key in url_masks.keys():
    for mask in url_masks[key].get('paged_list_formats',[]):
        page_masks.append({
            'site':key,
            'mask':mask,
        })

In [77]:
df_masks = pd.DataFrame(page_masks)

In [78]:
page_sites=['www.francesoir.fr',
'www.francetvinfo.fr',
'www.breizh-info.com',
'lesmoutonsrebelles.com',
'lemediaen442.fr',
'www.profession-gendarme.com',
'www.dreuz.info',
'www.lelibrepenseur.org',
'www.polemia.com',
'reseauinternational.net',
'www.mondialisation.ca',
'lesakerfrancophone.fr',
'lesalonbeige.fr',
'lesobservateurs.ch',
'lecourrier-du-soir.com',
'www.les-crises.fr',
'infodujour.fr',
'www.medias-presse.info',
'www.alnas.fr',
'www.bvoltaire.fr',
'ripostelaique.com',
'lalettrepatriote.com',
'www.la-petite-souris-normande.com',
'www.revue-elements.com',
'www.epochtimes.fr',
'extime.fr',
'elucid.media',
'www.fdesouche.com',
'qactus.fr',
'planetes360.fr',
'www.legorafi.fr']

In [79]:
for site in page_sites:
    if site not in df_masks['site'].values:
        print(site)

www.francesoir.fr
www.francetvinfo.fr
www.breizh-info.com
www.profession-gendarme.com
www.dreuz.info
www.lelibrepenseur.org
www.polemia.com
www.mondialisation.ca
lesakerfrancophone.fr
lesobservateurs.ch
infodujour.fr
www.medias-presse.info
www.la-petite-souris-normande.com
www.revue-elements.com
www.epochtimes.fr
extime.fr
elucid.media
www.legorafi.fr


In [76]:
df_masks

# Sauvegarde
---







In [16]:
df_sites.to_csv('data/tmp/230526-Sites.csv')